In [6]:
#!pip install textblob 
#!pip install wordcloud
#import nltk
#nltk.download('stopwords')


   ---------------------------------------- 0.0/300.2 kB ? eta -:--:--
   ----- ---------------------------------- 41.0/300.2 kB 1.9 MB/s eta 0:00:01
   --------------- ------------------------ 112.6/300.2 kB 1.3 MB/s eta 0:00:01
   ----------------------- ---------------- 174.1/300.2 kB 1.2 MB/s eta 0:00:01
   ------------------------------------ --- 276.5/300.2 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 300.2/300.2 kB 1.1 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [32]:
#Importing the necessary pachages
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')#Used for cleaner and more visualing appealing plots
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import warnings
warnings.filterwarnings('ignore')

In [8]:
df = pd.read_csv('vaccination_tweets.csv')#Reading the .csv file

In [9]:
df.info()#Displaying informtion about the data in the DatFrame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11020 entries, 0 to 11019
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                11020 non-null  int64 
 1   user_name         11020 non-null  object
 2   user_location     8750 non-null   object
 3   user_description  10340 non-null  object
 4   user_created      11020 non-null  object
 5   user_followers    11020 non-null  int64 
 6   user_friends      11020 non-null  int64 
 7   user_favourites   11020 non-null  int64 
 8   user_verified     11020 non-null  bool  
 9   date              11020 non-null  object
 10  text              11020 non-null  object
 11  hashtags          8438 non-null   object
 12  source            11019 non-null  object
 13  retweets          11020 non-null  int64 
 14  favorites         11020 non-null  int64 
 15  is_retweet        11020 non-null  bool  
dtypes: bool(2), int64(6), object(8)
memory usage: 1.2+ MB


In [10]:
df.isnull().sum()#Displaying the information of empty data

id                     0
user_name              0
user_location       2270
user_description     680
user_created           0
user_followers         0
user_friends           0
user_favourites        0
user_verified          0
date                   0
text                   0
hashtags            2582
source                 1
retweets               0
favorites              0
is_retweet             0
dtype: int64

In [11]:
df.columns#Checking the names of columns in the df

Index(['id', 'user_name', 'user_location', 'user_description', 'user_created',
       'user_followers', 'user_friends', 'user_favourites', 'user_verified',
       'date', 'text', 'hashtags', 'source', 'retweets', 'favorites',
       'is_retweet'],
      dtype='object')

In [12]:
text_df = df.drop(['id', 'user_name', 'user_location', 'user_description', 'user_created',
       'user_followers', 'user_friends', 'user_favourites', 'user_verified',
       'date', 'hashtags', 'source', 'retweets', 'favorites',
       'is_retweet'], axis=1)#Removing the un-necessary columns from the df
text_df.head()

,text
0,Same folks said daikon paste could treat a cyt...
1,While the world has been on the wrong side of ...
2,#coronavirus #SputnikV #AstraZeneca #PfizerBio...
3,"Facts are immutable, Senator, even when you're..."
4,Explain to me again why we need a vaccine @Bor...


In [13]:
text_df.info()#Deisplaying the info of the new df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11020 entries, 0 to 11019
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    11020 non-null  object
dtypes: object(1)
memory usage: 86.2+ KB


In [14]:
#Defining a processing function fr the newly formed datset
def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+https\S+", '',text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#','',text)
    text = re.sub(r'[^\w\s]','',text)
    text_tokens = word_tokenize(text)
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

In [16]:
# Applying the processing function to each element in the 'text' column
text_df['text'] = text_df['text'].apply(data_processing)

# Dropping duplicate rows based on the 'text' column
text_df = text_df.drop_duplicates('text')

# Initialize a PorterStemmer for stemming
stemmer = PorterStemmer()

#function for stemming that will be applied to each row of the 'text' column
def stemming(data):
    # Apply stemming to each word in the text
    stemmed_text = [stemmer.stem(word) for word in data]
    return stemmed_text

#USing the stemming function to each element in the 'text' column
text_df['text'] = text_df['text'].apply(lambda x: stemming(word_tokenize(x)))


In [17]:
text_df.head()#Displaying the new dataset after removing the duplicates

,text
0,"[folk, said, daikon, past, could, treat, cytok..."
1,"[world, wrong, side, histori, year, hope, bigg..."
2,"[coronaviru, sputnikv, astrazeneca, pfizerbion..."
3,"[fact, immut, senat, even, your, ethic, sturdi..."
4,"[explain, need, vaccin, borisjohnson, matthanc..."


In [18]:
text_df.info()#Info of the new dataset without duplicate values

<class 'pandas.core.frame.DataFrame'>
Index: 10543 entries, 0 to 11019
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    10543 non-null  object
dtypes: object(1)
memory usage: 164.7+ KB


In [20]:
def polarity(words_list):
    # Joins the list of words into a single string
    text = ' '.join(words_list)
    # Calculate polarity using TextBlob
    return TextBlob(text).sentiment.polarity

text_df['polarity'] = text_df['text'].apply(polarity)


In [21]:
def sentiment(label):#applying the sentiment and giving it numeric labels
    if label <0:
        return "Negative"
    elif label ==0:
        return "Neutral"
    elif label>0:
        return "Positive"
text_df['sentiment'] = text_df['polarity'].apply(sentiment)

In [25]:
# Convert the list of words back to a string in the 'text' column
text_df['text'] = text_df['text'].apply(lambda x: ' '.join(x))
# Use CountVectorizer on the modified 'text' column
vect = CountVectorizer(ngram_range=(1, 2)).fit(text_df['text'])


In [26]:
X = text_df['text']
Y = text_df['sentiment']
X = vect.transform(X)

In [27]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [36]:
model = LogisticRegression()
model.fit(x_train, y_train)
model_pred = model.predict(x_test)
model_acc = accuracy_score(model_pred, y_test)

In [37]:
print(confusion_matrix(y_test, model_pred))
print("\n")
print(classification_report(y_test, model_pred))

accuracy = accuracy_score(y_test, model_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")#Accuracy

[[  65   93   31]
 [   2 1299    1]
 [   7   74  537]]


              precision    recall  f1-score   support

    Negative       0.88      0.34      0.49       189
     Neutral       0.89      1.00      0.94      1302
    Positive       0.94      0.87      0.90       618

    accuracy                           0.90      2109
   macro avg       0.90      0.74      0.78      2109
weighted avg       0.90      0.90      0.89      2109

Accuracy: 90.14%
